# 03. Feature Engineering

**Proyecto:** Fantasy Bidding Intelligence  
**Objetivo:** Crear y transformar features para maximizar la señal predictiva de los modelos de clasificación y regresión.  
Esta fase se basa en la auditoría de datos y EDA realizado previamente.


### Librerías y Dataset
Importamos las librerías necesarias y cargamos el dataset unificado limpio.


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de pandas y seaborn
pd.set_option("display.max_columns", None)
sns.set(style="whitegrid")

# Cargar dataset
df = pd.read_csv("../data/processed/data_model.csv")
df.head()


,date,jugador,precio,posicionJugador,puntosJugador,equipoLiga,avgPoints,estado,variacion,jornada,detalles,equipo,ganancias,num_pujas
0,2025-10-29,A. ZAKHARYAN,0.205,3.0,7.0,16.0,1.8,0,NaN,11,2.0,NaN,NaN,0
1,2025-10-29,F. UGRINIC,0.170,3.0,8.0,19.0,2.0,0,NaN,11,2.0,NaN,NaN,0
2,2025-10-29,A. OSAMBELA,0.162,3.0,10.0,50.0,2.5,0,NaN,11,2.0,NaN,NaN,0
3,2025-10-29,A. SAN MIGUEL,0.198,1.0,0.0,4.0,0.0,0,NaN,11,2.0,NaN,NaN,0
4,2025-10-29,A. ITURBE,0.196,1.0,0.0,23.0,0.0,0,NaN,11,2.0,NaN,NaN,0


### Tipos de variables
Confirmamos los tipos de variables para planificar las transformaciones.


In [7]:
df.info()
df.describe(include="all").T


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304 entries, 0 to 303
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             304 non-null    object 
 1   jugador          304 non-null    object 
 2   precio           304 non-null    float64
 3   posicionJugador  304 non-null    float64
 4   puntosJugador    304 non-null    float64
 5   equipoLiga       304 non-null    float64
 6   avgPoints        304 non-null    float64
 7   estado           304 non-null    int64  
 8   variacion        269 non-null    float64
 9   jornada          304 non-null    int64  
 10  detalles         304 non-null    float64
 11  equipo           68 non-null     object 
 12  ganancias        68 non-null     float64
 13  num_pujas        304 non-null    int64  
dtypes: float64(8), int64(3), object(3)
memory usage: 33.4+ KB


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
date,304,25,2025-10-29,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN
jugador,304,209,A. OSAMBELA,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
precio,304.0,NaN,NaN,NaN,2.980204,3.661019,0.16,0.183,1.0515,5.65575,15.391
posicionJugador,304.0,NaN,NaN,NaN,2.582237,0.893586,1.0,2.0,3.0,3.0,4.0
puntosJugador,304.0,NaN,NaN,NaN,26.141447,18.977224,-6.0,10.75,23.0,37.0,77.0
equipoLiga,304.0,NaN,NaN,NaN,103.6875,283.637449,1.0,9.0,17.0,48.0,1370.0
avgPoints,304.0,NaN,NaN,NaN,3.042763,1.598188,-2.0,2.2,3.1,4.0,14.0
estado,304.0,NaN,NaN,NaN,0.279605,0.818986,0.0,0.0,0.0,0.0,5.0
variacion,269.0,NaN,NaN,NaN,-0.008758,0.064914,-0.211,-0.048,-0.01,0.03,0.171
jornada,304.0,NaN,NaN,NaN,13.230263,2.152504,11.0,12.0,13.0,13.0,18.0


### Transformaciones Numéricas
- Escalado y normalización de variables numéricas.
- Creación de features derivadas para eficiencia o momentum.


In [8]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

num_features = ["precio", "avgPoints", "puntosJugador", "variacion", "detalles"]

# Escalado estándar
scaler = StandardScaler()
df[num_features] = scaler.fit_transform(df[num_features])

# Feature derivada: eficiencia = avgPoints / precio
df["eficiencia"] = df["avgPoints"] / (df["precio"] + 1e-6)  # evitar división por 0

# Feature derivada: momentum = variacion binaria
df["subida"] = (df["variacion"] > 0).astype(int)

df.head()


,date,jugador,precio,posicionJugador,puntosJugador,equipoLiga,avgPoints,estado,variacion,jornada,detalles,equipo,ganancias,num_pujas,eficiencia,subida
0,2025-10-29,A. ZAKHARYAN,-0.759291,3.0,-1.010317,16.0,-0.778890,0,NaN,11,-1.019826,NaN,NaN,0,1.025813,0
1,2025-10-29,F. UGRINIC,-0.768867,3.0,-0.957535,19.0,-0.653542,0,NaN,11,-1.019826,NaN,NaN,0,0.850007,0
2,2025-10-29,A. OSAMBELA,-0.771056,3.0,-0.851972,50.0,-0.340172,0,NaN,11,-1.019826,NaN,NaN,0,0.441177,0
3,2025-10-29,A. SAN MIGUEL,-0.761206,1.0,-1.379788,4.0,-1.907022,0,NaN,11,-1.019826,NaN,NaN,0,2.505266,0
4,2025-10-29,A. ITURBE,-0.761754,1.0,-1.379788,23.0,-1.907022,0,NaN,11,-1.019826,NaN,NaN,0,2.503467,0


### Transformaciones Categóricas
Variables categóricas:
- `posicionJugador`  
- `estado`  
- `equipoLiga`  
- `jornada`  

Se aplicarán estrategias de encoding según la señal y frecuencia de categorías:
- One-hot encoding para pocas categorías frecuentes
- Target encoding o agrupación para categorías raras


In [9]:
# Confirmar categorías
categorical_features = ["posicionJugador", "estado", "equipoLiga", "jornada"]
for col in categorical_features:
    print(f"{col} unique values: {df[col].nunique()}")


posicionJugador unique values: 4
estado unique values: 6
equipoLiga unique values: 20
jornada unique values: 4


In [10]:
# One-hot para posición y estado (pocas categorías)
df = pd.get_dummies(df, columns=["posicionJugador", "estado"], drop_first=True)


In [11]:
# Target encoding: equipoLiga y jornada
# Se puede usar mean de recibe_puja
target = "recibe_puja"

# Target encoding suavizado (simple ejemplo)
for col in ["equipoLiga", "jornada"]:
    means = df.groupby(col)[target].mean()
    df[col + "_target_enc"] = df[col].map(means)

# Opcional: eliminar columnas originales si se prefiere usar solo target encoding
# df.drop(columns=["equipoLiga", "jornada"], inplace=True)


KeyError: 'Column not found: recibe_puja'

In [ ]:
df.head()
df.info()
